In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genaidataset2/model.py
/kaggle/input/genaidataset2/evaluate_models.py
/kaggle/input/genaidataset2/dataloader.py


In [2]:
!pip install datasets
!pip install sacremoses
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=694e253d35b111ba4f6d4eacc6e19cb84fc8ea61c030104adf0c2b5643a7387f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import os
import datetime

file_path = "/kaggle/working/ft_marian_final_100k_epoch_1.pt"  # Replace with your actual file
mod_time = os.path.getmtime(file_path)
print("Last modified:", datetime.datetime.fromtimestamp(mod_time))

Last modified: 2025-03-31 18:01:29.559567


In [3]:
import sys
sys.path.append("/kaggle/input/genaidataset2")

In [4]:
# import modules
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import MarianTokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

from model import NMTModel, Trainer
from dataloader import NMTDataset
from evaluate_models import calculate_scores

In [5]:
# load dataset
dataset = load_dataset('cfilt/iitb-english-hindi')

train_data = dataset['train']
train_data = train_data.select(range(1000000))
val_data = dataset['validation']

model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)

max_length = 75

train_dataset = NMTDataset(train_data, tokenizer, tokenizer, max_length)
val_dataset = NMTDataset(val_data, tokenizer, tokenizer, max_length)

README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

In [11]:
# initialize the model
model = NMTModel(model_name)
trainer = Trainer(model, train_dataset, val_dataset, epochs=6, batch_size=64, lr=3e-5)

In [12]:
model.load_state_dict(torch.load('/kaggle/working/ft_marian_final_100k.pt'))

<ipython-input-12-dfbcc2e08c72>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/ft_marian_final_100k.pt'))


<All keys matched successfully>

In [13]:
trainer = Trainer(model, train_dataset, val_dataset, epochs = 2, batch_size=64, lr=3e-5)

In [ ]:
# train the model
trainer.train()

Epoch 1/2: 100%|██████████| 15625/15625 [2:57:43<00:00,  1.47batch/s, loss=0.487]  


Epoch 1/2, Training Loss: 0.4628854981918335
Validation Loss: 1.22376772430208


Epoch 2/2:  72%|███████▏  | 11301/15625 [2:08:31<49:11,  1.47batch/s, loss=0.464]  